# Cobra Stability Tests
These stability tests will be run at both ASIAA (CobraCharmer) and Caltech (MSIM) to ensure that the statistical behavior of the cobras is consistent at both testing sites.

All tests will be run in the 3 non-colliding groups that ASIAA has historically used. The tests probe the following:
* Repeated move stability
* Subsequent move predictability

In [1]:
from importlib import reload
import numpy as np

from ics.cobraCharmer import pfi as pfiControl
reload(pfiControl)

<module 'ics.cobraCharmer.pfi' from '/Users/kadofong/work/pfscobras/ics_cobraCharmer/python/ics/cobraCharmer/pfi.py'>

In [8]:
stability_niter = 20
motormap_niter = 20

In [2]:
mod1Cobras = pfiControl.PFI.allocateCobraRange(range(1,2))
allCobras = mod1Cobras
oneCobra = pfiControl.PFI.allocateCobraList([(1,30)])    

# partition module 1 cobras into non-interfering sets
moduleCobras = {}
for group in 1,2,3:
    cm = range(group,58,3)
    mod = [1]*len(cm)
    
    moduleCobras[group] = pfiControl.PFI.allocateCobraList(zip(mod,cm))
    
group1Cobras = moduleCobras[1]
group2Cobras = moduleCobras[2]
group3Cobras = moduleCobras[3]

In [3]:
pfi = pfiControl.PFI(fpgaHost='localhost', doLoadModel=False) #'fpga' for real device.
#pfi.setFreq(allCobras)
pfi.loadModel('/Users/kadofong/work/pfscobras/jpltrip/data/updatedmap4.xml')

To test move stability, execute the following sequence for stage 2 motors (in both forward and reverse):
* Move to (CW/CCW) hardstop
* Move out (+/-)1000 steps
* Return to original position

In [15]:
# Stage 2 CW testing
for cobra_group in moduleCobras.values():
    for idx in range(stability_niter):
        pfi.homePhi ( cobra_group, dir='ccw' ) # is CCW hardstop positive or negative? Orientation question
        pfi.moveAllSteps(cobra_group, (0,1000), dirs=('cw','cw'))
        pfi.moveAllSteps(cobra_group, (0,100), dirs=('cw','cw'))
    pfi.homePhi ( cobra_group, dir='ccw' )

In [16]:
# Stage 2 CCW testing
for cobra_group in moduleCobras.values():
    for idx in range(stability_niter):
        pfi.homePhi ( cobra_group, dir='cw' ) # is CCW hardstop positive or negative? Orientation question
        pfi.moveAllSteps(cobra_group, (0,1000), dirs=('cw','ccw'))
        pfi.moveAllSteps(cobra_group, (0,100), dirs=('cw','ccw'))
    pfi.homePhi ( cobra_group, dir='cw' )

In [17]:
# Stage 1 CW testing:
for cobra_group in moduleCobras.values():
    pfi.moveAllSteps ( cobra_group, (0,1000), dirs=('ccw','ccw')) # move out phi motor for centroiding
    # do not have to be too conservative with step number,
    # because we are running with non-colliding groups
    for idx in range(stability_niter):
        pfi.homeTheta ( cobra_group, dir='ccw' ) # is CCW hardstop positive or negative? Orientation question
        pfi.moveAllSteps(cobra_group, (1000,0), dirs=('cw','cw'))
        pfi.moveAllSteps(cobra_group, (100,0), dirs=('cw','cw'))
    pfi.homeTheta ( cobra_group, dir='ccw' )
    pfi.homePhi ( cobra_group, dir='cw' )

In [18]:
# Stage 1 CCW testing:
for cobra_group in moduleCobras.values():
    pfi.moveAllSteps ( cobra_group, (0,1000), dirs=('ccw','ccw')) # move out phi motor for centroiding
    # do not have to be too conservative with step number,
    # because we are running with non-colliding groups
    for idx in range(stability_niter):
        pfi.homeTheta ( cobra_group, dir='cw' ) # is CCW hardstop positive or negative? Orientation question
        pfi.moveAllSteps(cobra_group, (1000,0), dirs=('ccw','cw'))
        pfi.moveAllSteps(cobra_group, (100,0), dirs=('ccw','cw'))
    pfi.homeTheta ( cobra_group, dir='cw' )